# Diagnostque de l'estimateur 

Le but de cette partie est se rendre compte de la fiabilité de l'estimateur de la variance en MLMC en calculant sa variance. On comparera les résultats à celle de l'estimateur classique MC. 

In [26]:
import numpy as np 

## calcul des estimateurs MLMC et MLMC log euclidien 
On se place dans la cas fidélité 2. Nos donnés sont des vecteurs gaussiens centrés réduits de dimension d.
formule de la variance: 
 
$\hat{\mathbb{V}}\left( f_1(X) \right) = \mathbb{E}\left[ f_1(X)^2 \right] $

In [27]:
def f_vec(X,v):
    return v.T@X

def f_vec_squared(X,v):
    return (v.T@X)**2

d = 10 # lenght of random vectors

v_1= np.array([-0.18268292, -0.63730044,  0.49158538, -0.51712638, -0.90171302,  0.40921266,  0.63791406,  0.70701292, -0.24888679,  0.94455888])#vecteur coef haute fidélité 
eps = np.random.normal(0,0.01,d) 
v_0 = v_1 + eps  #vecteur ceof basse fidélité 

In [28]:
def estim_var_MLMC(n0,n1):
    X1 = np.random.standard_normal((d,n1))
    X0 = np.random.standard_normal((d,n0))

    #estimateur de l'espérence
    Y1_1 = f_vec(X1,v_1)
    Y1_0 = f_vec(X1,v_0)
    Y0_0 = f_vec(X0,v_0)
    E_MLMC = np.mean(Y0_0) + np.mean(Y1_1-Y1_0)

    #estimateur de la variance 
    Y0_0_squared = f_vec_squared(X0,v_0)
    Y1_0_squared = f_vec_squared(X1,v_0)
    Y1_1_squared = f_vec_squared(X1,v_1)
    Var_MLMC = np.mean(Y0_0_squared) + np.mean(Y1_1_squared-Y1_0_squared)

    return Var_MLMC


In [29]:
def estim_var_MLMC_log(n0,n1):
    X1 = np.random.standard_normal((d,n1))
    X0 = np.random.standard_normal((d,n0))

    #estimateur de l'espérence
    Y1_1 = f_vec(X1,v_1)
    Y1_0 = f_vec(X1,v_0)
    Y0_0 = f_vec(X0,v_0)
    E_MLMC = np.mean(Y0_0) + np.mean(Y1_1-Y1_0)

    #estimateur de la variance 
    Y0_0_squared = f_vec_squared(X0,v_0)
    Y1_0_squared = f_vec_squared(X1,v_0)
    Y1_1_squared = f_vec_squared(X1,v_1)
    Var_MLMC_log = np.exp(np.log(np.mean(Y0_0_squared)) + np.log(np.mean(Y1_1_squared))-np.log(np.mean(Y1_0_squared)))
    return Var_MLMC_log

## Variance de ces estimateurs

On calcul maintenant la variance des rélutats que nous donne cet estimateur pour n=100000 résultats. 

In [30]:
def var_est_var_MLMC(n):
    n1=5
    n0=953
    tab_var=[]
    tab_var_log =[]
    for i in range(n):
        tab_var.append(estim_var_MLMC(n0,n1))
        tab_var_log.append(estim_var_MLMC_log(n0,n1))
    return [np.var(tab_var),np.var(tab_var_log)]

In [31]:
def estim_var_HF():
    d = 10 # lenght of random vectors

    v_1= np.array([-0.18268292, -0.63730044,  0.49158538, -0.51712638, -0.90171302,  0.40921266,  0.63791406,  0.70701292, -0.24888679,  0.94455888])#vecteur coef haute fidélité 
    eps = np.random.normal(0,0.01,d) 
    v_0 = v_1 + eps
    X1 = np.random.standard_normal((d,150))
    Y1_1_squared=f_vec_squared(X1,v_1)
    return(np.var(Y1_1_squared))
print (estim_var_HF())
    

26.013479932611038


In [32]:
print(var_est_var_MLMC(10000))

[0.031392035704323244, 0.031110678159381382]


##methode classique MC

In [33]:
def  var_est_var_MC(n):
    tab_var=[]
    d = 10
    v_1= np.array([-0.18268292, -0.63730044,  0.49158538, -0.51712638, -0.90171302,  0.40921266,  0.63791406,  0.70701292, -0.24888679,  0.94455888])

    for i in range(n):
        X = np.random.standard_normal((d,100))
        estim_var_MC=np.mean(f_vec_squared(X,v_1))
        tab_var.append(estim_var_MC)
    return (np.var(tab_var))


In [34]:
print(var_est_var_MLMC(10000))
print(var_est_var_MC(10000))

[0.0313201264645907, 0.0316268314391619]
0.2889094895121786


## Cas pathologiqe

In [38]:
def estim_var_MLMC_patho(n0,n1):
    X1 = np.random.standard_normal((d,n1))
    X0 = np.random.standard_normal((d,n0))

    #estimateur de l'espérence
    Y1_1 = f_vec(X1,v_1)
    Y1_0 = f_vec(X1,v_0)
    Y0_0 = f_vec(X0,v_0)
    E_MLMC = np.mean(Y0_0) + np.mean(Y1_1-Y1_0)

    #estimateur de la variance 
    Y0_0_squared = f_vec_squared(X0,v_0)
    Y1_0_squared = f_vec_squared(X1,v_0)
    Y1_1_squared = f_vec_squared(X1,v_1)
    mean = np.mean(Y1_1_squared-Y1_0_squared) #qu'est ce qui doit etre neg ? 
    if mean < 0:
        Var_MLMC = np.mean(Y0_0_squared) + mean
        Var_MLMC_log = np.exp(np.log(np.mean(Y0_0_squared)) + np.log(np.mean(Y1_1_squared))-np.log(np.mean(Y1_0_squared)))
        return [Var_MLMC, Var_MLMC_log]
    else :
        return 'a'


In [39]:
def var_est_var_MLMC_patho(n):
    n1=5
    n0=953
    tab_var=[]
    tab_var_log =[]
    for i in range(n):
        estim_var_patho = estim_var_MLMC_patho(n0,n1)
        if estim_var_patho != 'a' :
            tab_var.append(estim_var_patho[0])
            tab_var_log.append(estim_var_patho[1])

    print("Nombre de cas patho = ", len(tab_var))
    return [np.var(tab_var),np.var(tab_var_log)]

In [40]:
print(var_est_var_MLMC_patho(10000))

Nombre de cas patho =  0
[nan, nan]


c:\Users\Lili\Anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3747: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\Lili\Anaconda3\Lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\Lili\Anaconda3\Lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
